In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks/KGEmb")
!ls
!echo $(pwd)/data

Mounted at /content/drive
 application.log	 'lap_init FB237 32.npy'    optimizers
 colab_run.ipynb	 'lap_init FB237 500.npy'   requirements.txt
 data			  Lap_init.ipynb	    run.py
 data_loader.py		 'lap_init WN18RR 32.npy'   set_env.sh
 datasets		  Lap_No_Rships.ipynb	    test.py
 Lap_Emb.ipynb		  logs			    utils
'lap_init FB15K 32.npy'   models
/content/drive/MyDrive/Colab Notebooks/KGEmb/data


In [2]:
import numpy as np
from scipy import sparse
from sklearn.manifold import SpectralEmbedding
from scipy.spatial.distance import cdist
! pip install pyamg
! pip install fastlapmap
! pip install nmslib
from fastlapmap import LapEigenmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 5.0 MB/s 
     |████████████████████████████████| 188 kB 48.3 MB/s 


In [3]:
dataset = 'WN18RR'
dim = 500

entities=[] # create list to store vertex IDs

with open('data/' + dataset + '/train', 'r') as f:
    for line in f:
        edge = line.split()
        edge.remove(edge[1]) # remove relationship type
        if int(edge[0]) not in entities:
            entities.append(int(edge[0])) # add head ID
        if int(edge[1]) not in entities:
            entities.append(int(edge[1])) # add tail ID
    f.close()

mat = sparse.lil_matrix((max(entities)+1,max(entities)+1), dtype=np.int8).tocsr()

with open('data/' + dataset + '/train', 'r') as f:
    for line in f:
        edge = line.split()
        row_idx = entities.index(int(edge[0])) # get index for head entity
        col_idx = entities.index(int(edge[2])) # get index for tail entity
        mat[row_idx, col_idx] = 1 # add edge in adjacency matrix
        mat[col_idx, row_idx] = 1 # undirected graph so symetric adjacency matrix

embs = LapEigenmap(mat, n_eigs=dim, similarity='diffusion', norm_laplacian=False, k=10)
embs = np.asarray(embs)
np.save('lap_init ' + dataset + ' ' + str(dim), embs)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [4]:
# dim = 500

# embs = LapEigenmap(mat, n_eigs=dim, similarity='diffusion', norm_laplacian=False, k=10)
# embs = np.asarray(embs)
# np.save('lap_init ' + dataset + ' ' + str(dim), embs)

In [5]:
max(entities)

40942